## Edge ratio

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import zipfile
import os

# --- PARÁMETROS Y FUNCIONES ---
def process_indicators(csv_path, column_name):
    if not os.path.exists(csv_path):
        raise FileNotFoundError(f"El archivo '{csv_path}' no existe. Verifica la ruta base o el nombre del archivo.")
    df = pd.read_csv(csv_path, delimiter=';', encoding='utf-8')
    expanded = df.assign(**{column_name: df[column_name].str.split(',')}).explode(column_name)
    grouped = expanded.groupby(column_name).agg({
        'Edge Ratio': 'mean',
        'Ret/DD Ratio': 'mean',
        'Profit factor': 'mean',
        column_name: 'count'
    }).rename(columns={
        'Edge Ratio': 'Avg_EdgeRatio',
        'Ret/DD Ratio': 'Avg_RetDD',
        'Profit factor': 'Avg_ProfitFactor',
        column_name: 'Count'
    })
    grouped['Score'] = (
        grouped['Avg_EdgeRatio'] * 40 +
        grouped['Avg_RetDD'] * 15 +
        grouped['Avg_ProfitFactor'] * 15 +
        grouped['Count'] * 30
    )
    return grouped.sort_values(by='Score', ascending=False).head(15).index.tolist()

def matches_suffix(key, indicators_list):
    return any(key.endswith(f".{ind}") for ind in indicators_list)

def main():
    # Asegurarse de que la carpeta existe
    base_path = r"/mnt/c/Users/Administrador/Downloads"
    os.makedirs(base_path, exist_ok=True)
    # Archivos fijos dentro de la ruta proporcionada
    csv_path = os.path.join(base_path, "DatabankExport.csv")
    config_path = os.path.join(base_path, "config.xml")
    output_path = os.path.join(base_path, "Build Strategies.cfx")
    # Obtener los nombres de indicadores más prometedores
    top_entry_indicators = process_indicators(csv_path, 'Entry indicators')
    top_price_indicators = process_indicators(csv_path, 'Price indicators')
    top_exit_indicators = process_indicators(csv_path, 'Exit indicators')
    pf = pd.DataFrame({
        'Top Entry Indicators': top_entry_indicators,
        'Top Price Indicators': top_price_indicators,
        'Top Exit Indicators': top_exit_indicators
    })
    display(pf)
    essential_indicator_operators = [
        "CrossesAbove", "CrossesBelow", "IndicatorAboveMA", "IndicatorBelowMA",
        "IndicatorCrossesAboveMA", "IndicatorCrossesBelowMA",
        "IsRising", "IsFalling", "IsGreater", "IsGreaterOrEqual",
        "IsLower", "IsLowerOrEqual", "Equals", "NotEquals",
        "IsGreaterPercentil", "IsLowerPercentil", "IsGreaterCount", "IsLowerCount", "Not"
    ]

    # --- XML MODIFICACIÓN ---
    with zipfile.ZipFile(output_path, 'r') as zip_ref:
        zip_ref.extract("config.xml", path=base_path)
    tree = ET.parse(config_path)
    root = tree.getroot()
    blocks_root = root.find(".//BuildingBlocks")
    os.remove(config_path)

    if blocks_root is not None:
        for block in blocks_root.findall("Block"):
            key = block.attrib.get("key", "")
            category = block.attrib.get("category", "").lower()

            # Señales predefinidas
            if category == "signals":
                block.set("use", "true")
            
            # Entry indicators y operadores
            elif category == "indicators":
                if key in essential_indicator_operators:
                    block.set("use", "true")
                elif matches_suffix(key, top_entry_indicators):
                    block.set("use", "true")
                elif matches_suffix(key, top_exit_indicators):
                    block.set("use", "true")
                else:
                    block.set("use", "false")

            # Price indicators
            elif category == "stoplimitblocks":
                if matches_suffix(key, top_price_indicators):
                    block.set("use", "true")
                else:
                    block.set("use", "false")

    # Guardar XML temporal en la ruta base
    temp_xml = os.path.join(base_path, "config_temp.xml")
    tree.write(temp_xml, encoding="utf-8", xml_declaration=True)

    # Comprimir en .cfx dentro de la ruta base
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        zipf.write(temp_xml, arcname="config.xml")
    print(f"Archivo de configuración creado: {output_path}")
    # Eliminar el archivo temporal
    os.remove(temp_xml)

if __name__ == "__main__":
    main()

,Top Entry Indicators,Top Price Indicators,Top Exit Indicators
0,KAMA,ATR,CloseM
1,Low,BiggestRange,OpenM
2,HeikenAshiHigh,SmallestRange,HighM
3,ParabolicSAR,BBRange,LowM
4,HeikenAshiLow,BBWidthRatio,HighW
5,EMA,BarRange,OpenW
6,HeikenAshiClose,MTATR,LowW
7,KeltnerChannel,HighD,CloseW
8,Open,TrueRange,KAMA
9,HeikenAshiOpen,OpenD,CloseD


Archivo de configuración creado: /mnt/c/Users/Administrador/Downloads/Build Strategies.cfx


## Strategy Tester analysis

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def load_xls_report(filename):
    data = pd.read_excel(filename, header=None)
    start_index = data[data.iloc[:, 0]=='Transacciones'].index[0]
    end_index = data.iloc[start_index:, 0].isna().idxmax()
    columns = data.iloc[start_index+1].tolist()
    data.columns = columns
    balance = data['Beneficio'].iloc[start_index+2]
    data = data.iloc[start_index+3:end_index-1,:]
    resultados = []
    for i in range(0, len(data) - 1, 2):
        fila_in = data.iloc[i]
        fila_out = data.iloc[i + 1]
        if fila_in['Dirección'] == 'in' and fila_out['Dirección'] == 'out':
            precio_entrada = fila_in['Precio']
            precio_salida = fila_out['Precio']
            beneficio = fila_out['Beneficio'] - abs(fila_in['Comisión']) - abs(fila_out['Comisión'])
            balance += beneficio
            resultados.append({
                'Fecha': fila_in['Fecha/Hora'],
                'Símbolo': fila_in['Símbolo'],
                'Tipo': fila_in['Tipo'],
                'Precio Entrada': precio_entrada,
                'Precio Salida': precio_salida,
                'Volumen': fila_in['Volumen '],
                'Beneficio': beneficio,
                'Balance': balance
            })
    return pd.DataFrame(resultados)

filename='ReportTester-4000009439.xlsx'
# Statistics
df = load_xls_report(filename)
total_trades = df.shape[0]
profit_trades = df[df['Beneficio']>0].shape[0]
profit_avg = df[df['Beneficio']>0]['Beneficio'].mean()
loss_trades = df[df['Beneficio']<0].shape[0]
loss_avg = df[df['Beneficio']<0]['Beneficio'].mean()
beneficio_neto  = (profit_avg*profit_trades)+(loss_avg*loss_trades)
total_trades = profit_trades + loss_trades
expected_profit = beneficio_neto / total_trades
profit_avg_points = abs(df[df['Beneficio']>0]['Precio Entrada'] - df[df['Beneficio']>0]['Precio Salida']).mean() / 0.1
loss_avg_points = abs(df[df['Beneficio']<0]['Precio Entrada'] - df[df['Beneficio']<0]['Precio Salida']).mean() / 0.1
punto_equilibrio = abs(loss_avg)/(abs(loss_avg)+profit_avg)
win_rates = profit_trades/total_trades
balances = df['Balance']
cummax_balance = balances.cummax()
drawdowns = cummax_balance - balances
drawdown_percentages = drawdowns / cummax_balance
max_drawdown_percentage = drawdown_percentages.max() * 100

print(f"Número de operaciones totales: {total_trades}")
print(f"Número de operaciones ganadoras: {profit_trades}")
print(f"Número de operaciones perdedoras: {loss_trades}")
print(f"Media de puntos en operaciones ganadoras {profit_avg_points:.2f} puntos")
print(f"Media de puntos en operaciones perdedoras {loss_avg_points:.2f} puntos")
print(f"Ganancia media: {profit_avg:.2f}€")
print(f"Pérdida media: {loss_avg:.2f}€")
print(f"Rentabilidad esperada: {expected_profit:.4f}€")
print(f"Porcentaje de aciertos actual: {win_rates*100.0:.2f}%")
print(f"Porcentaje de aciertos para equilibrio: {punto_equilibrio*100.0:.2f}%")
print(f"El máximo drawdown es: {max_drawdown_percentage:.2f}%")

## Best Optimization Params

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

def parse_mt5_optimization_report(xml_file_path):
    """
    Parsea el reporte de optimización de MT5 exportado como XML (Excel 2003),
    y devuelve un DataFrame con todas las configuraciones y sus métricas.
    Versión mejorada con manejo de errores y compatibilidad con diferentes formatos MT5.
    """
    with open(xml_file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    soup = BeautifulSoup(content, features='xml')  # Usamos el parser XML nativo
    
    # Versión flexible para encontrar la hoja de resultados
    worksheet_names = ['Tester Optimizator Results', 'Optimization Results', 'Results']
    worksheet = None
    
    for name in worksheet_names:
        worksheet = soup.find('Worksheet', {'ss:Name': name}) or soup.find('worksheet', {'ss:name': name})
        if worksheet:
            break
    
    if worksheet is None:
        # Si no encontramos por nombre, buscamos cualquier hoja que contenga datos de optimización
        worksheets = soup.find_all(['Worksheet', 'worksheet'])
        for ws in worksheets:
            if ws.find('Row') or ws.find('row'):
                worksheet = ws
                break
    
    if worksheet is None:
        available_sheets = [ws.get('ss:Name') or ws.get('ss:name') for ws in soup.find_all(['Worksheet', 'worksheet'])]
        raise ValueError(f"No se encontró la hoja de resultados en el XML. Hojas disponibles: {available_sheets}")

    # Buscamos la tabla (con compatibilidad para mayúsculas/minúsculas)
    table = worksheet.find(['Table', 'table'])
    rows = table.find_all(['Row', 'row'], recursive=False)  # Buscamos solo filas directas
    
    if not rows:
        raise ValueError("No se encontraron filas de datos en la hoja de resultados")
    
    # Primera fila: nombres de columnas
    headers = []
    header_cells = rows[0].find_all(['Cell', 'cell'])
    for cell in header_cells:
        data = cell.find('Data', {'ss:Type': 'String'}) or cell.find('data')
        headers.append(data.get_text(strip=True) if data else f"Columna_{len(headers)+1}")
    
    # Resto de filas: datos
    data = []
    for row in rows[1:]:
        cells = row.find_all(['Cell', 'cell'])
        row_values = []
        for cell in cells:
            data_elem = (cell.find(['Data', 'data']) or 
                         cell.find('Data', {'ss:Type': 'Number'}) or 
                         cell.find('Data', {'ss:Type': 'String'}))
            row_values.append(data_elem.get_text(strip=True) if data_elem else '')
        data.append(row_values)
    
    # Convertimos a DataFrame
    df = pd.DataFrame(data, columns=headers)
    
    # Limpieza y conversión numérica
    df = df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
    df.replace('', pd.NA, inplace=True)
    df.dropna(how='all', inplace=True)
    
    # Columnas numéricas comunes en MT5 (ajusta si hace falta)
    numeric_cols = [
        "Pass", "Result", "Profit", "Expected Payoff", "Profit Factor",
        "Recovery Factor", "Sharpe Ratio", "Custom", "Equity DD %",
        "Trades", "max_orders", "orders_time_delay", "max_spread",
        "stoploss", "takeprofit"
    ]
    
    # Convertimos solo las columnas que existan
    numeric_cols = [col for col in numeric_cols if col in df.columns]
    for col in numeric_cols:
        # Soporte a comas como separador decimal
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')
    
    return df

def pareto_frontier(df, objectives):
    """
    Retorna un subconjunto del DataFrame 'df' que conforma el frente de Pareto
    bajo los objetivos indicados.

    'objectives' es una lista de tuplas (columna, sentido), 
    donde sentido puede ser 'max' o 'min'.
    """
    data = df.copy()

    # Convertimos todo a "maximizar" (lo que sea 'min' lo multiplicamos por -1).
    for col, sense in objectives:
        if sense == 'min':
            data[col] = -data[col]

    is_dominated = np.zeros(len(data), dtype=bool)

    # Revisamos si una fila i es dominada por otra fila j
    for i in range(len(data)):
        if is_dominated[i]:
            continue  # Ya está marcada como dominada

        for j in range(len(data)):
            if i == j:
                continue

            better_or_equal_j = True
            strictly_better_j = False

            for col, _ in objectives:
                if data.iloc[j][col] < data.iloc[i][col]:
                    better_or_equal_j = False
                    break
                elif data.iloc[j][col] > data.iloc[i][col]:
                    strictly_better_j = True

            if better_or_equal_j and strictly_better_j:
                is_dominated[i] = True
                break

    # Nos quedamos con las filas no dominadas
    pareto_df = df.loc[~is_dominated].copy()
    return pareto_df

def main():
    # 1. Cargamos el DataFrame
    xml_file = "/mnt/c/Users/Administrador/Downloads/ReportOptimizer-4842620.xml"
    df = parse_mt5_optimization_report(xml_file)
    
    # 2. Obtenemos el frente de Pareto (max Profit, max Sharpe, min Drawdown)
    pf = pareto_frontier(df, [
        ("Profit", "max"),
        ("Sharpe Ratio", "max"),
        ("Equity DD %", "min"),
        ("max_spread", "min")
    ])
    print(f"\nSe encontraron {len(pf)} configuraciones en el frente de Pareto.\n")

    print(pf[["Profit", "Sharpe Ratio", "Equity DD %", "max_orders", "orders_time_delay",
              "max_spread", "stoploss", "takeprofit"]].head(10))

if __name__ == "__main__":
    main()

## Arima best params searcher

In [ ]:
import os
import glob
import pandas as pd
import pmdarima as pm
import warnings
warnings.filterwarnings('ignore')

def determine_arima_params(series, start_p=0, start_q=0, max_p=5, max_q=5, m=1, seasonal=False, stepwise=True, suppress_warnings=True, trace=False):
    """
    Determina automáticamente los parámetros p, d, q para un modelo ARIMA dado una serie temporal.

    Args:
        series (pd.Series): Serie temporal de datos.
        start_p (int): Valor inicial de p para la búsqueda.
        start_q (int): Valor inicial de q para la búsqueda.
        max_p (int): Valor máximo de p para considerar.
        max_q (int): Valor máximo de q para considerar.
        m (int): Periodicidad para modelos estacionales. Por defecto es 1 (no estacional).
        seasonal (bool): Si True, busca modelos SARIMA.
        stepwise (bool): Si True, utiliza búsqueda stepwise para acelerar el proceso.
        suppress_warnings (bool): Si True, suprime advertencias durante el ajuste.
        trace (bool): Si True, imprime información detallada durante la búsqueda.

    Returns:
        tuple: (p, d, q) óptimos para el modelo ARIMA.
    """
    # Ajustar el modelo ARIMA automáticamente
    model = pm.auto_arima(
        series,
        start_p=start_p,
        start_q=start_q,
        max_p=max_p,
        max_q=max_q,
        m=m,
        seasonal=seasonal,
        trace=trace,
        error_action='ignore',
        suppress_warnings=suppress_warnings,
        stepwise=stepwise
    )
    
    # Obtener los parámetros óptimos
    p, d, q = model.order
    
    return p, d, q

def analyze_arima_by_year(df):
    """
    Divide el DataFrame por años y determina los valores p, d, q de ARIMA para cada año.

    Args:
        df (pd.DataFrame): DataFrame con una columna '<CLOSE>' y una columna de fecha.

    Returns:
        dict: Diccionario con los resultados por año.
    """
    # Convertir la columna de fecha a datetime
    df['<DATE>'] = pd.to_datetime(df['<DATE>'], infer_datetime_format=True)

    # Agregar una columna con el año
    df['Year'] = df['<DATE>'].dt.year

    # Diccionario para almacenar los resultados por año
    results = {}

    for year, group in df.groupby('Year'):
        print(f"Procesando el año: {year}")
        series = group['<CLOSE>']
        if len(series) > 10:  # Asegurar datos suficientes
            p, d, q = determine_arima_params(series, trace=True)
            results[year] = {'p': p, 'd': d, 'q': q}
            print(f"Año {year}: p={p}, d={d}, q={q}")
        else:
            print(f"Año {year}: No hay suficientes datos para ARIMA.")
    
    return results

# Ejemplo de uso:
if __name__ == "__main__":
    file_folder = r'/mnt/c/Users/Administrador/Downloads'
    file_pattern = os.path.join(file_folder, 'GDAXI_*.csv')
    df_file_path = glob.glob(file_pattern)
    df = pd.read_csv(df_file_path[0], delimiter='\t')
    print(df.head(1))
    print(df.tail(1))

    # Obtener los resultados por año
    results_by_year = analyze_arima_by_year(df)

    # Imprimir los resultados finales
    for year, params in results_by_year.items():
        print(f"Año {year}: p={params['p']}, d={params['d']}, q={params['q']}")
